---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def read_allhomes():
    allhomes=pd.read_csv('City_Zhvi_AllHomes.csv')
    return allhomes

def read_univtowns():
    univtowns=pd.read_csv('university_towns.txt', sep='/t')
    return univtowns
# read_univtowns()

def read_gdp_an():
    GDP_an=pd.read_excel('gdplev.xls', index_col=0, usecols=[0,1], header=5).dropna()
    return GDP_an
#read_gdp_an()

def read_gdp_qrt():
    GDP_qrt=pd.read_excel('gdplev.xls', index_col=4, header=5).drop(['GDP in billions of current dollars', 
                                                'Unnamed: 3', 'GDP in billions of chained 2009 dollars',
                                                'Unnamed: 2','Unnamed: 6'],axis=1).dropna()
    GDP_qrt=GDP_qrt.loc['2000q1':]
    return GDP_qrt

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

    values = []
    with open('university_towns.txt') as file:
        lines = file.readlines()

    for line in lines:
        if '[edit]' in line:
            current_state = line[:-7]
        else:
            line=line[:line.find('(')]
            values.append((current_state, line))

    df=pd.DataFrame(['State', 'RegionName'])
    df=pd.DataFrame(values, columns=['State', 'RegionName'])
    df['RegionName']=df['RegionName'].str.strip()
    #df=df.set_index(["State", "RegionName"])
    return df

get_list_of_university_towns().head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP_qrt=read_gdp_qrt()
    GDP_diff=GDP_qrt['GDP in billions of current dollars.1'].diff()
    
    for i, qrt in enumerate(GDP_diff.index):
        if (GDP_diff.iloc[i+1]<0) and (GDP_diff.iloc[i+2]<0):
            #print('Recession starts from {}'.format(qrt))
            return qrt
    return None

get_recession_start()

'2008q3'

In [6]:
# %matplotlib inline
# GDP_qrt=read_gdp_qrt()
# GDP_qrt.loc['2007q2':'2010q2'].plot()

In [7]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    GDP_qrt=read_gdp_qrt()
    GDP_qrt=GDP_qrt.loc[get_recession_start():]
    GDP_diff=GDP_qrt['GDP in billions of current dollars.1'].diff()
    for i, qrt in enumerate(GDP_diff.index):
        if (GDP_diff.iloc[i-1]>0) and (GDP_diff.iloc[i]>0):
            #print('Recession starts from {}'.format(qrt))
            return qrt
    return None
get_recession_end()

'2009q4'

In [8]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    GDP_qrt=read_gdp_qrt()
    start_quarter = get_recession_start()
    end_quarter = get_recession_end()
    return GDP_qrt.loc[start_quarter:end_quarter,'GDP in billions of current dollars.1'].idxmin()
get_recession_bottom()

'2009q2'

In [9]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    allhomes=pd.read_csv('City_Zhvi_AllHomes.csv')

    allhomes['State'].replace(states, inplace = True)
    allhomes=allhomes.set_index(['State','RegionName'])

    #start_todrop=allhomes.columns.get_loc('1996-04')#
    end_todrop=allhomes.columns.get_loc('1999-12')
    allhomes=allhomes.drop(list(allhomes.columns[0:end_todrop+1]),axis=1)

    def quarters(col):
        if col.endswith(("01", "02", "03")):
            s = col[:4] + "q1"
        elif col.endswith(("04", "05", "06")):
            s = col[:4] + "q2"
        elif col.endswith(("07", "08", "09")):
            s = col[:4] + "q3"
        else:
            s = col[:4] + "q4"
        return s

    column_quarters=[]
    for col in allhomes.columns.values:
        column_quarters.append(quarters(col))

    allhomes.columns=column_quarters
    allhomes=allhomes.groupby(allhomes.columns, axis=1).mean()

    return allhomes.sort_index()
convert_housing_data_to_quarters().shape

(10730, 67)

In [28]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    houses=convert_housing_data_to_quarters()

    recession_start_int=houses.columns.get_loc(get_recession_start())
    year_before = houses.columns[recession_start_int-1]
    bottom=get_recession_bottom()

    all_houses_price=pd.concat([houses[year_before],houses[bottom]], axis=1, keys=[year_before,bottom])
    all_houses_price['Price Ratio']=all_houses_price[year_before].div(all_houses_price[bottom])
    
    university_towns=get_list_of_university_towns()
    university_towns=university_towns.set_index(['State','RegionName'])

    university_houses = pd.merge(university_towns,all_houses_price, how='inner', left_index=True, right_index=True)
    non_university_houses=all_houses_price[~all_houses_price.index.isin(university_houses.index)]
    
    t_stat, p_value = ttest_ind(university_houses['Price Ratio'],
                                      non_university_houses['Price Ratio'], nan_policy='omit')
    p0=0.01
    if t_stat < p0:
        different=True
    else:
        different=False
       
    if t_stat < 0:
        better='university town'
    else:
        better='non-university town'    
    
    return different, p_value, better
run_ttest()

(True, 0.0027240637047531249, 'university town')

houses=convert_housing_data_to_quarters()

recession_start_int=houses.columns.get_loc(get_recession_start())
year_before = houses.columns[recession_start_int-1]
bottom=get_recession_bottom()

all_houses_price=pd.concat([houses[year_before],houses[bottom]], axis=1, keys=[year_before,bottom])
all_houses_price['Price Ratio']=all_houses_price[year_before].div(all_houses_price[bottom])
all_houses_price.shape


university_towns=get_list_of_university_towns()
university_towns=university_towns.set_index(['State','RegionName'])

university_houses = pd.merge(university_towns,all_houses_price, how='inner', left_index=True, right_index=True)

non_university_houses=all_houses_price[~all_houses_price.index.isin(university_houses.index)]
#university_housing = pd.merge(university_towns, housing, how = "inner",
                                 # left_index = True, right_index = True)
#non_university_housing = housing[~housing.index.isin(university_housing.index)]
#non_university_housing.head()
#university_houses.shape

print(non_university_houses.shape)
t_stat, p_value = ttest_ind(university_houses['Price Ratio'],
                                      non_university_houses['Price Ratio'], nan_policy='omit')
p0=0.01
if t_stat < p0:
    different=True
    better='university town'
else:
    different=False
    better='non-university town'
better

m1=university_houses['Price Ratio'].mean()
var1=(university_houses['Price Ratio'].std())**2
n1=len(university_houses)
m2=non_university_houses['Price Ratio'].mean()
var2=(non_university_houses['Price Ratio'].std())**2
n2=len(non_university_houses)
t_stat=(m1-m2)/np.sqrt(var1/n1+var2/n2)